# 1. 介绍
在深度学习中，需要大量的计算，我们经常被硬件条件所限制，所以在别人的模型基础上进行修改优化，成了很多人的选择，这相当于是一场接力赛，我们在著名的模型基础上，接过接力棒，做我们想做的事情。如果没有这个接力的过程，普通人想学习深度学习，并有所成果，时间和人力成本将高出很多。

本实验，我们提供了已经训练好的模型，并不进行进一步的训练，而是转向对训练成果进行评估。

本课将加载已有的模型，来对牙齿的根管治疗中的牙齿部分和根管部分来进行检测，然后将预测结果储存起来，并在图像上画出代表标注的矩形框。这是深度学习和医学的结合。

In [ ]:
import os
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

from dataset import voc
from retinanet import model, val
from retinanet import transforms as aug
%matplotlib inline

# 2. 初始化
## 2.1 设定常量
这些常量在接下来的程序中都不需要，也不能随便改动。

下面进行简单的解释：
1. root_path：数据根目录
2. img_path：图像路径
3. result_path：模型和预测结果的储存路径
4. model_name：要加载的模型名称
5. image_size：预测用的图像大小，我们要将原图变换到模型采用的大小
6. max_detections：一张图像中最多预测的标注数量
7. num_classes：总的类别数量，分别是牙齿和根管

In [ ]:
# consts
root_path = './data'
img_path = "./data/JPEGImages"
result_path = './result'
model_name = "root_treatment"

image_size=512
max_detections = 4
num_classes = 2

## 2.2 加载模型和数据
我们的任务是进行预测（evaluate），所以将数据集data_set命名为val_set。这里val_set是一个可迭代对象，里面加载了`./data/ImageSets/Main/val_list.txt`中记录的数据，其中图像从`img_path`加载，人工的标注从`./data/Annotation`中加载。

同时，图像在加载的过程中，会通过`val_trans`的一系列变换，依次进行下面的操作：
1. 补全图像为正方形
2. 将图像修改为长宽512大小
3. 修改图像色阶，对比度
4. 将图像转化成pytorch认可的tensor格式

In [ ]:
val_trans = aug.Compose([
    aug.Pad(),
    aug.Resize(image_size, image_size),
    aug.AutoLevel(min_level_rate=1, max_level_rate=1),
    aug.AutoContrast(),
    aug.Contrast(1.25),
    aug.ToTensor()
])

val_set = voc.VOCDetection(
    root_path,
    image_set="val_list",
    transforms=val_trans
)

我们对数据集进行观察。val_set可以直接取下标，我们选择第一个内容进行观察。

他的每一个元素都是(tensor, ([labels], [annnotations], {pad_loc, scale}))格式。

其中annnotations是一个list，里面有四个数字，分别代表了标注框的左上角横纵坐标，右下角横纵坐标。注意，我们的坐标计算都是以*最左上角为原点*的。

pad_loc记录了我们之前对图形进行的pad变换，这里的[70, 70, 0, 0]代表了上下都填充了70像素，而左右并没有改变。

scale记录了我们之前对图形进行的resize变化，这里的表示原图是新图的0.8258064516129032大小。注意这里说的原图，是已经被pad操作处理过的正方形图像。

In [ ]:
val_set[0][1]

对数据有了直观的概念后，我们加载模型root_treatment

In [ ]:
net = torch.load(os.path.join(result_path, model_name + ".pth"), map_location='cpu')
device = torch.device("cpu")
net = net.to(device)

## 2.3 进行预测
下面这个函数就是预测是关键了，我们在函数里面用注释来做讲解。这里介绍还没有介绍的参数含义：
1. score_threshold：网络预测结果的衡量标准。网络会对每一个预测进行评估，代表了他的自信程度，比如0.9代表他很自信这个结果是正确的，0.4代表他很不确定结果的正确性。我们在这里设置的目的是告诉网络，我们只需要看你觉得自信程度高于score_threshold的结果，其他的你自己留着
2. save_path：预测结果的保存路径

In [ ]:
def _get_detections(dataset, retinanet, num_classes=2, score_threshold=0.5, max_detections=4, save_path=None, model_name=None):
    # 准备一个可以容纳全部数据的多维list
    all_detections = [[None for i in range(num_classes)] for j in range(len(dataset))]
    
    # 模型初始化，这里有个参数，我们在第三个课程中会看到他的作用。
    retinanet.eval()
    retinanet.set_nms(0.7)
    with torch.no_grad():

        for index in range(len(dataset)):
            # data是我们之前看到的复杂结构 (img, ([labels], [gts], {scale, pad_loc...}))
            data = dataset[index]
            # get scale
            scale = data[1][2]["scale"]
            # get pad loc, up, down, left, right
            pad_loc = data[1][2]["pad_loc"]

            # 我们将图像作为参数传入神经网络中，他将返回三个值，分别是
            # score 网络对这次预测的自信程度，labels网络认为他的类别， boxes网络预测出的标注坐标
#             scores, labels, boxes = retinanet(data[0].permute(0, 1, 2).cuda().float().unsqueeze(dim=0))
            scores, labels, boxes = retinanet(data[0].permute(0, 1, 2).to(device).float().unsqueeze(dim=0))
            scores = scores.cpu().numpy()
            labels = labels.cpu().numpy()
            boxes  = boxes.cpu().numpy()
            
            # 由于模型需要用512的图像进行预测，所以这里我们为了得到原图，进行了一些反变换
            # correct boxes for image scale
            boxes /= scale
            for bbx in boxes:
                # height fix
                bbx[1] -= pad_loc[0]
                bbx[3] -= pad_loc[0]
                # width fix
                bbx[0] -= pad_loc[2]
                bbx[2] -= pad_loc[2]
            
            # 这里我们将选择score > score_threshold的预测结果。然后我们将结果写入all_detections中
            indices = np.where(scores > score_threshold)[0]
            if indices.shape[0] > 0:
                # select those scores
                scores = scores[indices]

                # find the order with which to sort the scores
                scores_sort = np.argsort(-scores)[:max_detections]

                # select detections
                image_boxes      = boxes[indices[scores_sort], :]
                image_scores     = scores[scores_sort]
                image_labels     = labels[indices[scores_sort]]
                image_detections = np.concatenate([image_boxes, np.expand_dims(image_scores, axis=1), np.expand_dims(image_labels, axis=1)], axis=1)
                                
                # copy detections to all_detections
                for label in range(num_classes):
                    all_detections[index][label] = image_detections[image_detections[:, -1] == label, :-1]
            else:
                # copy detections to all_detections
                for label in range(num_classes):
                    all_detections[index][label] = np.zeros((0, 5))
            
            # 我们每次处理完都进行一次输出，让我们对网络的预测进展有个直观的了解
            print("\r {}/{}".format(index + 1, len(dataset)), end='')
    
    # 将结果保存到save_path下的csv中，为了区分不同的csv，我们将传入的参数作为csv的名字的一部分
    if save_path:
        csv_data = pd.DataFrame(columns=['id', 'serial', 'class', 'x1', 'y1', 'x2', 'y2', 'prob'])
        csv_id = 0
        for index in range(len(all_detections)):
            image_name = dataset.images[index].split('/')[-1].split('.')[0]
            for label in range(len(all_detections[index])):
                for box in range(len(all_detections[index][label])):
                    bbx = all_detections[index][label][box]
                    csv_data.loc[csv_id] = [image_name, box, label, bbx[0], bbx[1], bbx[2], bbx[3], bbx[4]]
                    csv_id += 1
        file_name = "{}_{}".format(
            model_name,
            score_threshold,
        )
        csv_data.to_csv(os.path.join(save_path, file_name+".csv"))
    
    return all_detections

接下来，我们就可以指定score_threshold，让网络输出他的结果了。我们将结果放在模型的同级目录下。

In [ ]:
score_thresholds = [0.5, 0.55, 0.6]
score_threshold = score_thresholds[0]
all_detections = _get_detections(val_set, net, num_classes=num_classes, score_threshold=score_threshold, max_detections=max_detections, save_path=result_path, model_name=model_name)

# 3. 观察结果
将结果保存到文件中是方便我们以后的观察，当图像少的时候，比如这里我们只有49张，我们可以很快预测完，但是当图像数量太多的时候，需要运行很久。这时候，如果已经保存了之前的结果，那么我们不需要再运行便可以进行观察，或者进行其他的操作了。

In [ ]:
def draw_box(img_pd, img_path=None):
    # 接受一个 pandas 的 dataframe，以及原始图像的路径
    # 因为dataframe中，每一行代表一个预测框，而我们的处理单位是一张图，一张图上最多有4个框。所以这里将所有的图像名字取出
    name_series = img_pd.drop_duplicates(subset='id', inplace=False)['id']
    
    for image_name in name_series:
        # 按照图像名称进行迭代，选择图像名称为image_name的全部框
        img_some = img_pd[img_pd['id'] == image_name]
        
        # 如果是只想看类型为0的框，可以自行选择将下面一行取消注释
#         img_some = img_some[img_some['class'] == 0]
        
        img = Image.open(os.path.join(img_path, image_name + ".jpg"))
        np_img = np.asarray(img)
        
        # 对于每一个图中的框，将其画在图像上，颜色 RGB = [0, 255, 0]，粗细为 2 像素。这里的画指的是修改图像的三维矩阵。
        for index in range(img_some.shape[0]):
            img_detect = img_some.iloc[index]
        
            img_id = img_detect['id']
            label = img_detect['class']
            x1 = img_detect['x1']
            y1 = img_detect['y1']
            x2 = img_detect['x2']
            y2 = img_detect['y2']
            bbs = []
            bbs.append(BoundingBox(x1=x1, y1=y1, x2=x2, y2=y2))
            bbs_on_img = BoundingBoxesOnImage(bbs, shape=np_img.shape)
            np_img = bbs_on_img.draw_on_image(np_img, size=2, color=[0, 255, 0])
        
        # 最终得到一个numpy array 格式的三维矩阵，可以直接画出来
        ia.imshow(np_img)

这里我们载入已经保存了的预测结果。可以看到我们的结果。class表示类型，serial表示这是这张图类型class的第几个预测框，prob表示网络的自行程度。

In [ ]:
detection = "{}_{}".format(
            model_name,
            score_threshold,
        )
img_pd = pd.read_csv(os.path.join(result_path, detection+".csv"), index_col=0)
img_pd.head(5)

然后我们就可以调用之前的函数来观察绘画的结果了。可以看到，在 img_path 下的原始图像上，出现了很多矩形框，这些框将牙齿部分和根管部分框了起来，其精确度相当的高，这就是神经网络的威力了。

In [ ]:
draw_box(img_pd, img_path=img_path)